Copyright 2021 Floq authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
#@title Copyright 2021 Floq authors, All Rights Reserved.
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

In [ ]:
!pip install floq_client --quiet

In [ ]:
# Imports
import numpy as np
import sympy

import cirq
import floq.client

## Floq simulation

In [ ]:
nrows = 10
ncols = 2
qubits = cirq.GridQubit.rect(nrows, ncols) # 20 qubits
parameters = sympy.symbols([f'a{idx}' for idx in range(nrows * ncols)])
circuit = cirq.Circuit(cirq.HPowGate(exponent=p).on(q) for p, q in zip(parameters, qubits))

### New observable compatible with Floq

Floq accepts observables in the type of `cirq.ops.linear_combinations.PauliSum` only

In [ ]:
observables = []
for i in range(nrows):
  for j in range(ncols):
    if i < nrows - 1:
      observables.append(cirq.Z(qubits[i*ncols + j]) * cirq.Z(qubits[(i + 1)*ncols + j]))
      # Z[i * ncols + j] * Z[(i + 1) * ncols + j]
    if j < ncols - 1:
      observables.append(cirq.Z(qubits[i*ncols + j]) * cirq.Z(qubits[i*ncols + j+1]))
      # Z[i * ncols + j] * Z[i * ncols + (j + 1)]
len(observables)

28

In [ ]:
import copy

def sum_pauli_strings(obs):
  m = copy.deepcopy(obs[0])
  for o in obs[1:]:
    m += o
  return m

def split_observables(obs):
# hack: split observables into many buckets with at most 26 terms
  obs_buckets = [obs[s:s+25] for s in range(0, len(obs), 25)]
  measure = []
  for obs in obs_buckets:
    measure.append(sum_pauli_strings(obs))
  return measure

In [ ]:
measure = split_observables(observables)

In [ ]:
[len(m) for m in measure]

[25, 3]

In [ ]:
# These two results should have the same number of Pauli string terms
assert sum_pauli_strings(observables) == sum_pauli_strings(measure)

### Padding qubits

Because Floq's minimum number of qubits is 26, we need to pad it. This will be changed in the future.

In [ ]:
def pad_circuit(circ, qubits):
    return circ + cirq.Circuit([cirq.I(q) for q in qubits])

def get_pad_qubits(circ):
  num = len(circ.all_qubits())
  return [cirq.GridQubit(num, pad) for pad in range(26 - num)]

In [ ]:
pad_qubits = get_pad_qubits(circuit)
padded_circuit = pad_circuit(circuit, pad_qubits)

In [ ]:
padded_circuit

(0, 0): ────H^a0────────

(0, 1): ────H^a1────────

(1, 0): ────H^a2────────

(1, 1): ────H^a3────────

(2, 0): ────H^a4────────

(2, 1): ────H^a5────────

(3, 0): ────H^a6────────

(3, 1): ────H^a7────────

(4, 0): ────H^a8────────

(4, 1): ────H^a9────────

(5, 0): ────H^a10───────

(5, 1): ────H^a11───────

(6, 0): ────H^a12───────

(6, 1): ────H^a13───────

(7, 0): ────H^a14───────

(7, 1): ────H^a15───────

(8, 0): ────H^a16───────

(8, 1): ────H^a17───────

(9, 0): ────H^a18───────

(9, 1): ────H^a19───────

(20, 0): ───────────I───

(20, 1): ───────────I───

(20, 2): ───────────I───

(20, 3): ───────────I───

(20, 4): ───────────I───

(20, 5): ───────────I───

In [ ]:
values = np.random.random(len(parameters))
resolver = {s: v for s, v in zip(parameters, values)}
print(resolver)

## Using Floq simulator

Before going further, please **FORK THIS COLAB NOTEBOOK**, and **DO NOT SHARE YOUR API KEY WITH OTHERS PLEASE**

### Create & start a Floq instance

In [ ]:
# Please specify your API_KEY
API_KEY = "" #@param {type:"string"}

In [ ]:
!floq-client "$API_KEY" worker start

In [ ]:
client = floq.client.CirqClient(API_KEY)

### Expectation values from the circuit and measurements

In [ ]:
energy = client.simulator.simulate_expectation_values(padded_circuit, measure, resolver)

In [ ]:
# energy shows expectation values on each Pauli sum in measure.
energy

In [ ]:
# Here is the total energy
sum(energy)

### Samples from the circuit

In [ ]:
niter = 100
samples = client.simulator.run(padded_circuit, resolver, niter)

In [ ]:
samples

### Stop the Floq instance

In [ ]:
!floq-client "$API_KEY" worker stop